In [1]:
from pytdx.hq import TdxHq_API
from pytdx.params import TDXParams
from datetime import datetime, timedelta
import pandas as pd
import polars as pl
import os
from tqdm import tqdm

In [3]:
symbol = '688076'

df = pl.read_parquet(f'cn_data_daily_bar/data/{symbol}.parquet')
df.head()

open,close,high,low,vol,amount,datetime
f64,f64,f64,f64,f64,f64,str
83.0,77.5,83.15,75.0,3.1114182e7,2.4463e9,"""2021-05-20 15:00"""
79.03,78.0,80.0,75.01,1.3552022e7,1.0578e9,"""2021-05-21 15:00"""
77.32,78.35,80.32,73.35,1.1815883e7,9.10502528e8,"""2021-05-24 15:00"""
77.5,78.74,79.9,75.51,8.395213e6,6.4973728e8,"""2021-05-25 15:00"""
79.99,77.51,85.58,77.51,1.063303e7,8.6297824e8,"""2021-05-26 15:00"""


In [4]:
dts = []
for dt_str in df['datetime']:
    dts.append(datetime.strptime(dt_str.split(' ')[0], '%Y-%m-%d'))

df['datetime'] = dts
df.head()

open,close,high,low,vol,amount,datetime
f64,f64,f64,f64,f64,f64,datetime
83.0,77.5,83.15,75.0,3.1114182e7,2.4463e9,2021-05-20 00:00:00
79.03,78.0,80.0,75.01,1.3552022e7,1.0578e9,2021-05-21 00:00:00
77.32,78.35,80.32,73.35,1.1815883e7,9.10502528e8,2021-05-24 00:00:00
77.5,78.74,79.9,75.51,8.395213e6,6.4973728e8,2021-05-25 00:00:00
79.99,77.51,85.58,77.51,1.063303e7,8.6297824e8,2021-05-26 00:00:00


In [5]:
ls = df['low']
df['pivot_low'] = ((ls <= ls.shift(1)).fill_null(False) & (ls <= ls.shift(-1)).fill_null(False))
df['pivot_low_price'] = df['pivot_low'] * df['low']
df['pivot_low_price'] = df['pivot_low_price'].apply(lambda x: x if x > 0 else None)

hs = df['high']
df['pivot_high'] = ((hs >= hs.shift(1)).fill_null(False) & (hs >= hs.shift(-1)).fill_null(False))
df['pivot_high_price'] = df['pivot_high'] * df['high']
df['pivot_high_price'] = df['pivot_high_price'].apply(lambda x: x if x > 0 else None)

In [6]:
df['freq'] = ['1d'] * len(df)
df['symbol'] = [symbol] * len(df)

In [6]:
df.to_parquet(f'../analyzer.view.server/dump/cn_pivot/compute/{symbol}.1d.parquet')

- Trend Logic
    - 向前找 valid pivot
        - Lower High
        - Higher Low
    - 持续多久
    - 两个valid pivot间间隔多少
        - period
        - pivot
        - price diff
        - revert trend
    - Trend Break
    - 当前 Bar/价
        - Potentially making High/Low
        - Potentially continuing trend
        - Potentially breaking trend
    - 如果趋势延续,预测下一个 pivot 的位置
    - Multi-periods
- Feature
    - 趋势持续的周期
    - 趋势刚开始 / 已持续长时间
    - Reversal
    - Chanel
- Strategy
    - 趋势反转
    - 趋势启动 -> Chanel
- Visual